In [ ]:
# ============================================================================
# CELL 1: INSTALAÇÃO COMPLETA
# ============================================================================

import sys
print(f"Python version: {sys.version}")

# Instalar dependências
print("\n[INFO] Instalando dependências...\n")
!pip install -q google-adk>=1.18.0
!pip install -q google-cloud-bigquery>=3.15.0
!pip install -q google-cloud-bigquery-storage>=2.26.0
!pip install -q scipy>=1.11.0
!pip install -q pandas>=2.1.0
!pip install -q numpy>=1.24.0

print("\n[OK] Todas as dependências instaladas!")

In [ ]:
# ============================================================================
# CELL 2: CONFIGURAÇÃO KAGGLE E LOGGING
# ============================================================================

import os
import logging
from kaggle_secrets import UserSecretsClient

# Configurar logging estruturado
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# API Key do Gemini (obrigatória)
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    logger.info("Gemini API key configurada com sucesso")
    print("[OK] Gemini API key configurada")
except Exception as e:
    logger.error(f"Erro ao configurar Gemini API key: {e}")
    print(f"[ERRO] {e}")
    print("\n[INFO] Configure em: Add-ons → Secrets → GOOGLE_API_KEY")
    raise

# BigQuery Service Account (opcional)
BIGQUERY_ENABLED = False
try:
    BIGQUERY_CREDENTIALS = UserSecretsClient().get_secret("BIGQUERY_SERVICE_ACCOUNT_JSON")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/tmp/bigquery_credentials.json"
    with open("/tmp/bigquery_credentials.json", "w") as f:
        f.write(BIGQUERY_CREDENTIALS)
    logger.info("BigQuery credentials configuradas")
    print("[OK] BigQuery credentials configuradas")
    BIGQUERY_ENABLED = True
except Exception as e:
    logger.warning(f"BigQuery não configurado: {e}")
    print("[AVISO] BigQuery não configurado (opcional)")
    print("\n[INFO] Para habilitar BigQuery:")
    print("   1. Crie Service Account no GCP")
    print("   2. Baixe JSON das credenciais")
    print("   3. Adicione em Kaggle Secrets: BIGQUERY_SERVICE_ACCOUNT_JSON")

In [ ]:
# ============================================================================
# CELL 3: IMPORTS COMPLETOS
# ============================================================================

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search

# BigQuery (se disponível)
if BIGQUERY_ENABLED:
    from google.adk.tools.bigquery import (
        BigQueryToolset,
        BigQueryCredentialsConfig,
        BigQueryToolConfig,
        WriteMode
    )
    from google.cloud import bigquery
    import google.auth

# Code Execution (verificar disponibilidade)
CODE_EXECUTION_AVAILABLE = False
try:
    from google.adk.tools.code_execution import code_execution
    CODE_EXECUTION_AVAILABLE = True
    logger.info("Code Execution disponível")
    print("[OK] Code Execution disponível")
except ImportError:
    logger.warning("Code Execution não disponível nesta versão do ADK")
    print("[AVISO] Code Execution não disponível nesta versão do ADK")

import pandas as pd
import numpy as np
from scipy import stats
import math
import json
import re
from typing import Dict, Any, List, Optional
from io import StringIO
import warnings

warnings.filterwarnings('ignore')
logger.info("Todos os imports carregados")
print("\n[OK] Imports carregados!")

In [ ]:
# ============================================================================
# CELL 4: STATISTICAL TOOLKIT (CENTRALIZADO)
# ============================================================================

class StatisticalToolkit:
    """Centraliza todas as ferramentas estatísticas em uma classe."""
    
    @staticmethod
    def calculate_sample_size(
        baseline_rate: float,
        mde: float,
        alpha: float = 0.05,
        power: float = 0.8
    ) -> Dict[str, Any]:
        """Calcula tamanho de amostra para experimento A/B.
        
        Args:
            baseline_rate: Taxa de conversão atual (ex: 0.025 = 2.5%)
            mde: Minimum Detectable Effect em pontos percentuais (ex: 0.5)
            alpha: Significância estatística (padrão: 0.05)
            power: Poder estatístico (padrão: 0.8)
        
        Returns:
            Dict com resultados completos
        """
        logger.info(f"Calculando sample size: baseline={baseline_rate}, mde={mde}")
        
        p1 = baseline_rate
        p2 = baseline_rate + (mde / 100)
        
        z_alpha = stats.norm.ppf(1 - alpha / 2)
        z_beta = stats.norm.ppf(power)
        
        numerator = (z_alpha + z_beta) ** 2 * (p1 * (1 - p1) + p2 * (1 - p2))
        denominator = (p1 - p2) ** 2
        
        n_per_group = math.ceil(numerator / denominator)
        
        result = {
            "sample_size_per_group": n_per_group,
            "total_sample_size": n_per_group * 2,
            "baseline_rate": baseline_rate,
            "target_rate": p2,
            "mde_percentage": mde,
            "mde_absolute": p2 - p1,
            "alpha": alpha,
            "power": power,
            "recommendation": f"Necessário {n_per_group:,} usuários por grupo (total: {n_per_group*2:,})",
            "estimated_weeks": {
                "1000_daily": math.ceil(n_per_group * 2 / (1000 * 7)),
                "5000_daily": math.ceil(n_per_group * 2 / (5000 * 7)),
                "10000_daily": math.ceil(n_per_group * 2 / (10000 * 7))
            }
        }
        
        logger.info(f"Sample size calculado: {n_per_group} por grupo")
        return result
    
    @staticmethod
    def calculate_statistical_significance(
        control_conversions: int,
        control_total: int,
        treatment_conversions: int,
        treatment_total: int,
        alpha: float = 0.05
    ) -> Dict[str, Any]:
        """Calcula significância estatística entre grupos A/B.
        
        Returns:
            Dict com p-value, uplift, intervalos
        """
        logger.info(f"Calculando significance: control={control_conversions}/{control_total}, "
                   f"treatment={treatment_conversions}/{treatment_total}")
        
        p1 = control_conversions / control_total if control_total > 0 else 0
        p2 = treatment_conversions / treatment_total if treatment_total > 0 else 0
        
        # Z-test para proporções
        p_pooled = (control_conversions + treatment_conversions) / (control_total + treatment_total)
        se = math.sqrt(p_pooled * (1 - p_pooled) * (1/control_total + 1/treatment_total))
        
        z = (p2 - p1) / se if se > 0 else 0
        p_value = 2 * (1 - stats.norm.cdf(abs(z)))
        
        # Uplift relativo e absoluto
        uplift_relative = ((p2 - p1) / p1 * 100) if p1 > 0 else 0
        uplift_absolute = (p2 - p1) * 100  # em pontos percentuais
        
        # Intervalo de confiança
        se_diff = math.sqrt(p1 * (1 - p1) / control_total + p2 * (1 - p2) / treatment_total)
        ci_margin = stats.norm.ppf(1 - alpha/2) * se_diff
        ci_lower = p2 - p1 - ci_margin
        ci_upper = p2 - p1 + ci_margin
        
        # Interpretação
        is_significant = p_value < alpha
        is_positive = p2 > p1
        
        if is_significant and is_positive:
            recommendation = "[OK] IMPLEMENTAR: Resultado significativo e positivo"
        elif is_significant and not is_positive:
            recommendation = "[AVISO] NÃO IMPLEMENTAR: Resultado significativo mas negativo"
        else:
            recommendation = "[INFO] CONTINUAR TESTANDO: Resultado não significativo"
        
        result = {
            "control_rate": p1,
            "treatment_rate": p2,
            "uplift_relative_percentage": uplift_relative,
            "uplift_absolute_pp": uplift_absolute,
            "p_value": p_value,
            "is_significant": is_significant,
            "is_positive": is_positive,
            "z_statistic": z,
            "confidence_interval_95": {
                "lower": ci_lower,
                "upper": ci_upper,
                "lower_pp": ci_lower * 100,
                "upper_pp": ci_upper * 100
            },
            "interpretation": "SIGNIFICATIVO (p < 0.05)" if is_significant else "NÃO SIGNIFICATIVO",
            "recommendation": recommendation,
            "sample_sizes": {
                "control": control_total,
                "treatment": treatment_total,
                "total": control_total + treatment_total
            }
        }
        
        logger.info(f"Significance calculado: p-value={p_value:.4f}, significant={is_significant}")
        return result
    
    @staticmethod
    def analyze_csv_dataframe(csv_data: str) -> Dict[str, Any]:
        """Analisa dados CSV em formato string.
        
        Args:
            csv_data: String com dados CSV
        
        Returns:
            Dict com estatísticas descritivas
        """
        logger.info("Analisando CSV dataframe")
        
        df = pd.read_csv(StringIO(csv_data))
        
        # Estatísticas numéricas
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        numeric_summary = {}
        if numeric_cols:
            for col in numeric_cols:
                numeric_summary[col] = {
                    "mean": float(df[col].mean()),
                    "median": float(df[col].median()),
                    "std": float(df[col].std()),
                    "min": float(df[col].min()),
                    "max": float(df[col].max()),
                    "q25": float(df[col].quantile(0.25)),
                    "q75": float(df[col].quantile(0.75))
                }
        
        # Valores faltantes
        missing = df.isnull().sum()
        missing_pct = (missing / len(df) * 100).round(2)
        missing_summary = {
            col: {"count": int(missing[col]), "percentage": float(missing_pct[col])}
            for col in df.columns if missing[col] > 0
        }
        
        # Duplicatas
        duplicate_rows = df.duplicated().sum()
        
        # Tipos de dados
        dtypes_summary = {col: str(dtype) for col, dtype in df.dtypes.items()}
        
        result = {
            "shape": {"rows": len(df), "columns": len(df.columns)},
            "columns": df.columns.tolist(),
            "dtypes": dtypes_summary,
            "missing_values": missing_summary,
            "duplicate_rows": int(duplicate_rows),
            "numeric_summary": numeric_summary,
            "sample_data": df.head(5).to_dict('records'),
            "memory_usage_mb": float(df.memory_usage(deep=True).sum() / 1024**2)
        }
        
        logger.info(f"CSV analisado: {len(df)} linhas, {len(df.columns)} colunas")
        return result


# Wrapper functions para FunctionTool (mantém compatibilidade com ADK)
def calculate_sample_size(baseline_rate: float, mde: float, alpha: float = 0.05, power: float = 0.8) -> str:
    """Wrapper que retorna JSON string para FunctionTool."""
    try:
        result = StatisticalToolkit.calculate_sample_size(baseline_rate, mde, alpha, power)
        return json.dumps(result, indent=2)
    except Exception as e:
        logger.error(f"Erro ao calcular sample size: {e}")
        return json.dumps({"error": str(e)})

def calculate_statistical_significance(
    control_conversions: int,
    control_total: int,
    treatment_conversions: int,
    treatment_total: int
) -> str:
    """Wrapper que retorna JSON string para FunctionTool."""
    try:
        result = StatisticalToolkit.calculate_statistical_significance(
            control_conversions, control_total, treatment_conversions, treatment_total
        )
        return json.dumps(result, indent=2)
    except Exception as e:
        logger.error(f"Erro ao calcular significance: {e}")
        return json.dumps({"error": str(e)})

def analyze_csv_dataframe(csv_data: str) -> str:
    """Wrapper que retorna JSON string para FunctionTool."""
    try:
        result = StatisticalToolkit.analyze_csv_dataframe(csv_data)
        return json.dumps(result, indent=2)
    except Exception as e:
        logger.error(f"Erro ao analisar CSV: {e}")
        return json.dumps({"error": str(e)})


# Criar FunctionTools
sample_size_tool = FunctionTool(
    function=calculate_sample_size,
    description="""Calcula tamanho de amostra necessário para experimento A/B.
    
    Parâmetros:
    - baseline_rate: Taxa atual (0.025 = 2.5%)
    - mde: Efeito mínimo detectável em pontos percentuais (0.5 = 0.5pp)
    - alpha: Significância (padrão 0.05)
    - power: Poder estatístico (padrão 0.8)
    
    Retorna JSON com sample_size_per_group, total_sample_size e estimativa de semanas."""
)

significance_tool = FunctionTool(
    function=calculate_statistical_significance,
    description="""Calcula significância estatística entre grupos A/B (z-test para proporções).
    
    Parâmetros:
    - control_conversions: Número de conversões no controle
    - control_total: Total de usuários no controle
    - treatment_conversions: Número de conversões no tratamento
    - treatment_total: Total de usuários no tratamento
    
    Retorna JSON com p-value, uplift relativo/absoluto, intervalo de confiança e recomendação."""
)

csv_analysis_tool = FunctionTool(
    function=analyze_csv_dataframe,
    description="""Analisa dados CSV fornecidos como string.
    
    Retorna estatísticas descritivas completas:
    - Shape (linhas, colunas)
    - Tipos de dados
    - Valores faltantes (contagem e %)
    - Linhas duplicadas
    - Estatísticas numéricas (média, mediana, quartis)
    - Amostra dos dados (5 primeiras linhas)"""
)

logger.info("StatisticalToolkit + FunctionTools criados")
print("[OK] StatisticalToolkit + 3 FunctionTools criados:")
print("   - calculate_sample_size")
print("   - calculate_statistical_significance")
print("   - analyze_csv_dataframe")

In [ ]:
# ============================================================================
# CELL 5: CONFIGURAÇÃO BIGQUERY (SE DISPONÍVEL)
# ============================================================================

bq_toolset = None

if BIGQUERY_ENABLED:
    try:
        # Opção 1: Service Account (para Kaggle)
        from google.oauth2 import service_account
        credentials = service_account.Credentials.from_service_account_file(
            "/tmp/bigquery_credentials.json"
        )
        
        credentials_config = BigQueryCredentialsConfig(credentials=credentials)
        tool_config = BigQueryToolConfig(write_mode=WriteMode.BLOCKED)
        
        bq_toolset = BigQueryToolset(
            credentials_config=credentials_config,
            bigquery_tool_config=tool_config
        )
        
        logger.info("BigQueryToolset configurado com sucesso")
        print("[OK] BigQueryToolset configurado!")
        print("   - Modo: BLOCKED (somente leitura)")
        print("   - Ferramentas: execute_sql, ask_data_insights")
        
    except Exception as e:
        logger.error(f"Erro ao configurar BigQuery: {e}")
        print(f"[AVISO] Erro ao configurar BigQuery: {e}")
        bq_toolset = None
        BIGQUERY_ENABLED = False
else:
    logger.info("BigQuery não disponível")
    print("[AVISO] BigQuery não disponível")
    print("\n[INFO] Para habilitar BigQuery:")
    print("   1. Crie Service Account no GCP")
    print("   2. Baixe JSON das credenciais")
    print("   3. Adicione em Kaggle Secrets: BIGQUERY_SERVICE_ACCOUNT_JSON")

In [ ]:
# ============================================================================
# CELL 6: SUB-AGENTES ESPECIALIZADOS
# ============================================================================

MODEL = "gemini-2.0-flash-exp"  # Modelo verificado e disponível
logger.info(f"Usando modelo: {MODEL}")

# 1. FUNNEL AGENT
funnel_tools = [google_search, csv_analysis_tool]
if bq_toolset:
    funnel_tools.append(bq_toolset)

funnel_agent = Agent(
    name="FunnelAgent",
    model=MODEL,
    instruction="""
    Você é o FunnelAgent - Especialista em Análise de Funil de Conversão.
    
    RESPONSABILIDADES:
    1. Analisar dados de funil (visita → cadastro → compra)
    2. Calcular taxas de conversão entre etapas
    3. Identificar pontos de maior drop-off
    4. Segmentar por canal, dispositivo, campanha
    
    FERRAMENTAS DISPONÍVEIS:
    - analyze_csv_dataframe: para dados CSV
    - BigQuery execute_sql: para dados em BigQuery (se disponível)
    - google_search: para benchmarks de mercado
    
    FORMATO DE OUTPUT (sempre JSON estruturado):
    {
      "etapas": [
        {"nome": "visita", "usuarios": 5000, "taxa_conversao": 100},
        {"nome": "signup", "usuarios": 3500, "taxa_conversao": 70},
        {"nome": "purchase", "usuarios": 1050, "taxa_conversao": 21}
      ],
      "gargalo_principal": "signup → purchase (queda de 70% para 30%)",
      "insights": ["insight 1", "insight 2"]
    }
    
    Seja objetivo e baseie suas conclusões em dados.
    """,
    tools=funnel_tools,
    output_key="funnel_report"
)

# 2. STATS AGENT
stats_agent = Agent(
    name="StatsAgent",
    model=MODEL,
    instruction="""
    Você é o StatsAgent - Especialista em Análise Estatística.
    
    RESPONSABILIDADES:
    1. Validar significância estatística de experimentos
    2. Calcular intervalos de confiança
    3. Interpretar p-values e uplift
    
    FERRAMENTAS:
    - calculate_statistical_significance: para testes A/B
    
    SEMPRE interprete os resultados estatísticos de forma clara,
    explicando se o experimento teve sucesso ou não.
    
    Use linguagem acessível para analistas não-estatísticos.
    """,
    tools=[significance_tool],
    output_key="stats_results"
)

# 3. INSIGHTS AGENT
insights_agent = Agent(
    name="InsightsAgent",
    model=MODEL,
    instruction="""
    Você é o InsightsAgent - Especialista em Recomendações de Growth.
    
    CONTEXTO:
    Você receberá dados de outros agentes via {funnel_report} e {stats_results}.
    
    RESPONSABILIDADES:
    1. Sintetizar análises anteriores
    2. Identificar quick wins (alto impacto, baixo esforço)
    3. Priorizar usando RICE: Reach × Impact × Confidence / Effort
    
    FORMATO OUTPUT (sempre estruturado):
    ## TOP 3 QUICK WINS
    1. [Nome] - Impacto: Alto | Esforço: Baixo | Score RICE: 8.5
    
    ## TOP 2 EXPERIMENTOS
    1. [Hipótese] - Métrica: [X] | Impacto estimado: +15%
    
    Seja específico e acionável.
    """,
    tools=[google_search],
    output_key="insights"
)

# 4. DATA QUALITY AGENT
data_quality_tools = [csv_analysis_tool]
if bq_toolset:
    data_quality_tools.append(bq_toolset)

data_quality_agent = Agent(
    name="DataQualityAgent",
    model=MODEL,
    instruction="""
    Você é o DataQualityAgent - Especialista em Validação de Dados.
    
    CHECKLIST DE VALIDAÇÃO:
    1. Missing Values: % de dados faltantes por coluna
    2. Duplicatas: IDs duplicados, eventos repetidos
    3. Consistência Temporal: eventos em ordem lógica
    4. Outliers: valores absurdos (ex: idade = 999)
    5. Integridade: todos os user_ids têm eventos esperados
    
    OUTPUT:
    - Score de qualidade: 0-100
    - Lista de problemas encontrados
    - Recomendações de correção (priorize as mais críticas)
    
    Use a ferramenta analyze_csv_dataframe para inspecionar os dados.
    """,
    tools=data_quality_tools,
    output_key="data_quality_report"
)

# 5. TRACKING AGENT
tracking_agent = Agent(
    name="TrackingAgent",
    model=MODEL,
    instruction="""
    Você é o TrackingAgent - Especialista em Validação de Tracking.
    
    VALIDAÇÕES:
    1. Eventos Esperados vs Encontrados
       - Ex: [page_view, signup, purchase] → Todos presentes?
    2. Schema Validation: campos obrigatórios presentes?
    3. Lógica de Negócio: usuário pode comprar sem signup?
    4. Cobertura: % de usuários com tracking completo
    
    OUTPUT:
    - Eventos encontrados: [lista]
    - Eventos faltando: [lista]
    - Gaps críticos: [descrição]
    - Recomendações de implementação
    
    Seja específico sobre quais eventos estão faltando e onde.
    """,
    tools=data_quality_tools,
    output_key="tracking_report"
)

# 6. EXPERIMENT AGENT
experiment_agent = Agent(
    name="ExperimentAgent",
    model=MODEL,
    instruction="""
    Você é o ExperimentAgent - Especialista em Design de Experimentos A/B.
    
    FRAMEWORK COMPLETO:
    1. HIPÓTESE: "Se [mudança], então [resultado] porque [razão]"
    2. MÉTRICA PRIMÁRIA: North Star Metric (ex: taxa de conversão)
    3. MÉTRICAS SECUNDÁRIAS: métricas de suporte
    4. GUARDRAILS: métricas que não podem piorar (ex: churn)
    5. RANDOMIZAÇÃO: como alocar usuários (por user_id, session, etc)
    6. SAMPLE SIZE: use calculate_sample_size tool
    7. DURAÇÃO: com base no tráfego diário
    8. ANÁLISE: método estatístico (t-test, chi-square)
    9. STOPPING RULES: quando parar (ex: p < 0.05 por 3 dias)
    
    SEMPRE use a ferramenta calculate_sample_size para calcular
    o tamanho de amostra necessário.
    
    OUTPUT: Documento completo pronto para implementação.
    """,
    tools=[sample_size_tool, google_search],
    output_key="experiment_plan"
)

logger.info("6 sub-agentes criados com sucesso")
print("[OK] 6 sub-agentes criados com instruções completas!")



In [ ]:
# ============================================================================
# CELL 7: LOOP AGENT - REFINAMENTO ITERATIVO
# ============================================================================

# Função de saída do loop
def exit_refinement_loop() -> str:
    """Chame esta função quando o refinamento estiver completo e aprovado."""
    logger.info("Exit refinement loop chamado")
    return json.dumps({"status": "approved", "message": "Refinamento completo"})

exit_loop_tool = FunctionTool(
    function=exit_refinement_loop,
    description="Chame quando o insight/experimento estiver refinado e aprovado para sair do loop."
)

# Agente Crítico (avalia qualidade)
critic_agent = Agent(
    name="CriticAgent",
    model=MODEL,
    instruction="""
    Você é um crítico rigoroso de planos de experimentos.
    
    Revise o experimento em {experiment_plan} e verifique:
    1. Hipótese está clara e testável?
    2. Métricas estão bem definidas?
    3. Sample size foi calculado?
    4. Duração é realista?
    5. Há guardrails para proteger experiência do usuário?
    
    IMPORTANTE: Se TUDO estiver perfeito, responda EXATAMENTE: "APPROVED"
    (pode ter texto antes ou depois, mas a palavra APPROVED deve aparecer)
    
    Caso contrário, liste os problemas específicos para correção.
    """,
    tools=[],
    output_key="critique"
)

# Agente Refinador (corrige com base na crítica)
refiner_agent = Agent(
    name="RefinerAgent",
    model=MODEL,
    instruction="""
    Você refina planos de experimento com base em feedback.
    
    ENTRADA:
    - Experimento atual: {experiment_plan}
    - Crítica recebida: {critique}
    
    LÓGICA:
    - Se critique contém "APPROVED" (case-insensitive), você DEVE chamar exit_refinement_loop
    - Caso contrário, reescreva o experimento corrigindo os problemas
    
    Sempre melhore o experimento incorporando todo o feedback.
    """,
    tools=[exit_loop_tool, sample_size_tool],
    output_key="experiment_plan"  # Sobrescreve o plano original
)

# Loop de refinamento
refinement_loop = LoopAgent(
    name="ExperimentRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=3  # Máximo 3 iterações de refinamento
)

logger.info("LoopAgent criado para refinamento iterativo")
print("[OK] LoopAgent criado para refinamento iterativo!")
print("   - CriticAgent avalia qualidade")
print("   - RefinerAgent corrige problemas")
print("   - Max 3 iterações")

In [ ]:
# ============================================================================
# CELL 8: AGENTES COMPOSTOS - PARALLEL & SEQUENTIAL
# ============================================================================

# PARALLEL: Diagnósticos independentes simultâneos
parallel_diagnostic = ParallelAgent(
    name="ParallelDiagnosticAgent",
    sub_agents=[funnel_agent, data_quality_agent, tracking_agent]
)

# SEQUENTIAL: Pipeline completo de análise
sequential_pipeline = SequentialAgent(
    name="SequentialAnalysisPipeline",
    sub_agents=[
        parallel_diagnostic,  # Etapa 1: Diagnósticos em paralelo
        stats_agent,          # Etapa 2: Análise estatística
        insights_agent,       # Etapa 3: Gerar insights
        experiment_agent,     # Etapa 4: Desenhar experimento
        refinement_loop       # Etapa 5: Refinar experimento
    ]
)

logger.info("Agentes compostos criados")
print("[OK] Agentes compostos criados!")
print("   - ParallelDiagnosticAgent: 3 análises simultâneas")
print("   - SequentialAnalysisPipeline: pipeline de 5 etapas + loop")

In [ ]:
# ============================================================================
# CELL 9: AGENTE COORDENADOR (ROOT)
# ============================================================================

coordinator_tools = [
    AgentTool(agent=funnel_agent),
    AgentTool(agent=stats_agent),
    AgentTool(agent=insights_agent),
    AgentTool(agent=data_quality_agent),
    AgentTool(agent=tracking_agent),
    AgentTool(agent=experiment_agent),
    google_search,
    csv_analysis_tool
]

if bq_toolset:
    coordinator_tools.append(bq_toolset)

coordinator = Agent(
    name="CoordinatorAgent",
    model=MODEL,
    instruction="""
    Você é o Growth & Experimentation - Marketing Data Scientist Specialist Partner.
    
    ## IDENTIDADE
    Cientista de dados sênior focado em:
    - Aquisição e ativação de usuários
    - Otimização de conversão
    - Design e análise de experimentos A/B
    - Retenção e lifetime value
    
    Você combina rigor analítico com pensamento criativo de growth.
    
    ## PRINCÍPIOS FUNDAMENTAIS
    1. **Data-Driven:** Decisões sempre baseadas em dados e hipóteses testáveis
    2. **Questione:** Pergunte quando houver ambiguidade
    3. **Priorize:** Quick wins + experimentos de alto impacto
    4. **Rigor Estatístico:** Valide significância sempre
    5. **Profundidade:** Vá além do óbvio, busque insights não-intuitivos
    6. **Transparência:** Explique raciocínio e mostre cálculos
    
    ## SUB-AGENTES DISPONÍVEIS (via AgentTool)
    Você pode chamar estes especialistas:
    
    - **FunnelAgent:** Analisa funil de conversão, identifica gargalos
      → Retorna: {funnel_report}
      
    - **StatsAgent:** Testes estatísticos, p-values, intervalos de confiança
      → Retorna: {stats_results}
      
    - **InsightsAgent:** Gera recomendações priorizadas e quick wins
      → Retorna: {insights}
      
    - **DataQualityAgent:** Valida qualidade e consistência dos dados
      → Retorna: {data_quality_report}
      
    - **TrackingAgent:** Verifica implementação de eventos/tracking
      → Retorna: {tracking_report}
      
    - **ExperimentAgent:** Desenha experimentos A/B completos
      → Retorna: {experiment_plan}
    
    ## FLUXO DE TRABALHO
    
    ### 1. INTAKE & UNDERSTANDING
    - Entenda e resuma o problema do usuário
    - Identifique dados disponíveis
    - Clarifique objetivos e métricas
    
    ### 2. SANITY CHECKS
    - Chame DataQualityAgent para validar dados
    - Chame TrackingAgent para verificar eventos
    - Identifique gaps críticos
    
    ### 3. EXPLORATORY ANALYSIS
    - Chame FunnelAgent para análise de funil
    - Use StatsAgent para métricas atuais
    - Identifique baseline e oportunidades
    
    ### 4. HYPOTHESIS GENERATION
    - Chame InsightsAgent para gerar hipóteses
    - Priorize usando RICE/ICE framework
    - Valide viabilidade técnica
    
    ### 5. EXPERIMENT DESIGN
    - Chame ExperimentAgent para experimentos
    - Valide sample size e duração
    - Defina métricas primárias e guardrails
    
    ### 6. DELIVERABLE
    - Executive Summary (3 bullets máximo)
    - Technical Analysis (com dados e cálculos)
    - Action Plan (priorizado)
    
    ## USANDO OUTPUT_KEYS
    
    Após chamar um sub-agente, você pode referenciar sua saída:
    
    Exemplo:
    1. Chame FunnelAgent → saída vai para {funnel_report}
    2. Você pode usar: "Segundo o {funnel_report}, o gargalo é..."
    3. Chame StatsAgent → saída vai para {stats_results}
    4. Combine: "Com base em {funnel_report} e {stats_results}..."
    
    ## FORMATO DE RESPOSTA FINAL
    
    ```
    # EXECUTIVE SUMMARY
    - [Bullet 1: Principal finding]
    - [Bullet 2: Oportunidade crítica]
    - [Bullet 3: Próxima ação recomendada]
    
    # ANÁLISE TÉCNICA
    [Justificativas, dados, cálculos, referências a {output_keys}]
    
    # ACTION PLAN
    ## Quick Wins (0-2 semanas)
    1. [Ação] - Impacto: [X] | Esforço: [Y]
    
    ## Experimentos (2-6 semanas)
    1. [Experimento] - Hipótese: [H] | Métrica: [M] | Sample size: [N]
    ```
    
    Seja objetivo, acionável e sempre baseado em dados.
    """,
    tools=coordinator_tools
)

logger.info("Coordenador criado com sucesso")
print("[OK] Coordenador criado com instruções completas!")
print("   - 6 sub-agentes via AgentTool")
print("   - Google Search + CSV Analysis")
if bq_toolset:
    print("   - BigQuery habilitado")

In [ ]:
# ============================================================================
# CELL 10: RUNNER
# ============================================================================

runner = InMemoryRunner(agent=coordinator)
logger.info("InMemoryRunner configurado")
print("\n[OK] InMemoryRunner configurado!")
print("\n" + "="*70)
print("Sistema completo pronto para uso!")
print("="*70)
print("\n[INFO] Componentes implementados:")
print("   [x] 6 Sub-agentes especializados")
print("   [x] Padrão LLM Coordinator")
print("   [x] ParallelAgent para diagnósticos")
print("   [x] SequentialAgent para pipeline")
print("   [x] LoopAgent para refinamento")
print("   [x] StatisticalToolkit centralizado")
print("   [x] FunctionTool para estatística")
print("   [x] CSV analysis tool")
print("   [x] Error handling robusto")
print("   [x] Logging estruturado")
if bq_toolset:
    print("   [x] BigQueryToolset configurado")
print("\n[OK] 100% da Arquitetura MDS Partner implementada!")

In [ ]:
# ============================================================================
# CELL 11: TESTE 1 - FERRAMENTAS ESTATÍSTICAS
# ============================================================================

print("\n" + "="*70)
print("TESTE 1: Ferramentas Estatísticas")
print("="*70)

try:
    # Sample size
    result1 = calculate_sample_size(baseline_rate=0.025, mde=0.5)
    print("\n[TEST] 1. Sample Size Calculation:")
    print(json.dumps(json.loads(result1), indent=2))
    
    # Significance
    result2 = calculate_statistical_significance(
        control_conversions=250,
        control_total=10000,
        treatment_conversions=280,
        treatment_total=10000
    )
    print("\n[TEST] 2. Statistical Significance:")
    print(json.dumps(json.loads(result2), indent=2))
    
    print("\n[OK] Ferramentas estatísticas funcionando!")
    
except Exception as e:
    logger.error(f"Erro nos testes estatísticos: {e}")
    print(f"\n[ERRO] Testes falharam: {e}")

In [ ]:
# ============================================================================
# CELL 12: TESTE 2 - CRIAR DADOS DE EXEMPLO
# ============================================================================

print("\n" + "="*70)
print("TESTE 2: Criando dados de exemplo")
print("="*70)

try:
    np.random.seed(42)
    n_users = 5000
    
    events = []
    channels = ['organic', 'paid_search', 'social', 'direct']
    devices = ['mobile', 'desktop']
    
    for i in range(n_users):
        user_id = f"user_{i}"
        channel = np.random.choice(channels)
        device = np.random.choice(devices)
        
        # Landing (100%)
        events.append({
            'user_id': user_id,
            'event': 'landing_page_view',
            'timestamp': f'2024-11-{np.random.randint(1,16):02d}',
            'device': device,
            'channel': channel
        })
        
        # Signup (70%)
        if np.random.random() < 0.7:
            events.append({
                'user_id': user_id,
                'event': 'signup',
                'timestamp': f'2024-11-{np.random.randint(1,16):02d}',
                'device': device,
                'channel': channel
            })
            
            # Purchase (30% dos signups)
            if np.random.random() < 0.3:
                events.append({
                    'user_id': user_id,
                    'event': 'purchase',
                    'timestamp': f'2024-11-{np.random.randint(1,16):02d}',
                    'device': device,
                    'channel': channel
                })
    
    df_events = pd.DataFrame(events)
    
    # Calcular métricas
    total = df_events['user_id'].nunique()
    signups = df_events[df_events['event'] == 'signup']['user_id'].nunique()
    purchases = df_events[df_events['event'] == 'purchase']['user_id'].nunique()
    
    logger.info(f"Dataset criado: {len(df_events)} eventos, {total} usuários")
    print(f"\n[OK] Dataset criado:")
    print(f"   - {len(df_events):,} eventos")
    print(f"   - {total:,} usuários únicos")
    print(f"   - Signups: {signups:,} ({signups/total*100:.1f}%)")
    print(f"   - Purchases: {purchases:,} ({purchases/total*100:.1f}%)")
    
    print(f"\n[INFO] Primeiras 10 linhas:")
    print(df_events.head(10).to_string())
    
    # Salvar para uso nos testes
    csv_data = df_events.to_csv(index=False)
    
except Exception as e:
    logger.error(f"Erro ao criar dados de exemplo: {e}")
    print(f"\n[ERRO] Falha ao criar dados: {e}")
    csv_data = None

In [ ]:
# ============================================================================
# CELL 13: TESTE 3 - CONSULTA SIMPLES AO COORDENADOR
# ============================================================================

print("\n" + "="*70)
print("TESTE 3: Consulta simples ao Coordenador")
print("="*70)

query1 = """
Quais são os 3 principais erros que empresas cometem ao analisar 
funis de conversão? Forneça exemplos práticos e como evitá-los.
"""

print(f"\n[QUERY] {query1.strip()}")
print("\n[INFO] Processando...\n")

try:
    response1 = await runner.run_debug(query1)
    print("="*70)
    print("RESPOSTA DO COORDENADOR")
    print("="*70)
    print(response1)
    print("\n[OK] Teste 3 concluído!")
    
except Exception as e:
    logger.error(f"Erro ao executar query 1: {e}")
    print(f"\n[ERRO] Falha na execução: {e}")
    print("\n[INFO] Verifique:")
    print("   1. GOOGLE_API_KEY está configurada corretamente")
    print("   2. Internet está habilitada no Kaggle")
    print("   3. Modelo está disponível")

In [ ]:
# ============================================================================
# CELL 14: TESTE 4 - ANÁLISE COMPLETA COM DADOS
# ============================================================================

if csv_data:
    print("\n" + "="*70)
    print("TESTE 4: Análise completa com dados CSV")
    print("="*70)
    
    query2 = f"""
    Analise o seguinte dataset de eventos de usuários e forneça:
    1. Diagnóstico completo do funil de conversão
    2. Identificação de gargalos principais
    3. Proposta de 2 experimentos A/B prioritários
    
    Dados CSV:
    ```
    {csv_data[:2000]}...
    ```
    
    Use todos os agentes necessários para uma análise completa.
    """
    
    print(f"\n[QUERY] Análise completa com {len(df_events)} eventos")
    print("\n[INFO] Processando (pode levar 2-3 minutos)...\n")
    
    try:
        response2 = await runner.run_debug(query2)
        print("="*70)
        print("ANÁLISE COMPLETA")
        print("="*70)
        print(response2)
        print("\n[OK] Teste 4 concluído!")
        
    except Exception as e:
        logger.error(f"Erro ao executar query 2: {e}")
        print(f"\n[ERRO] Falha na análise: {e}")
else:
    print("\n[AVISO] Teste 4 pulado (dados não disponíveis)")


In [ ]:
# ============================================================================
# CELL 15: TESTE 5 - CÁLCULO DE SAMPLE SIZE
# ============================================================================

print("\n" + "="*70)
print("TESTE 5: Interação direta com ExperimentAgent")
print("="*70)

query3 = """
Preciso desenhar um experimento A/B para melhorar a taxa de conversão
de signup para purchase, que atualmente está em 2.5%.

Quero detectar um aumento de pelo menos 0.5 pontos percentuais (de 2.5% para 3.0%).

Calcule:
1. Tamanho de amostra necessário
2. Duração estimada com 1000 usuários por dia
3. Desenhe o plano completo do experimento
"""

print(f"\n[QUERY] {query3.strip()}")
print("\n[INFO] Processando...\n")

try:
    response3 = await runner.run_debug(query3)
    print("="*70)
    print("PLANO DE EXPERIMENTO")
    print("="*70)
    print(response3)
    print("\n[OK] Teste 5 concluído!")
    
except Exception as e:
    logger.error(f"Erro ao executar query 3: {e}")
    print(f"\n[ERRO] Falha no planejamento: {e}")

In [ ]:
# ============================================================================
# CELL 16: FUNÇÃO HELPER PARA USO INTERATIVO
# ============================================================================

async def ask_growth_agent(question: str, verbose: bool = True) -> str:
    """
    Função helper para interagir com o Growth Agent de forma simples.
    
    Args:
        question: Pergunta ou solicitação para o agente
        verbose: Se True, mostra logs detalhados
    
    Returns:
        Resposta do agente como string
    
    Exemplo:
        response = await ask_growth_agent("Analise meu funil de conversão")
        print(response)
    """
    try:
        if verbose:
            logger.info(f"Processando pergunta: {question[:100]}...")
            print(f"\n[INFO] Processando sua pergunta...\n")
        
        response = await runner.run_debug(question)
        
        if verbose:
            logger.info("Resposta gerada com sucesso")
            print("\n[OK] Resposta gerada!\n")
        
        return response
        
    except Exception as e:
        logger.error(f"Erro ao processar pergunta: {e}")
        error_msg = f"[ERRO] Não foi possível processar sua pergunta: {e}"
        
        if verbose:
            print(f"\n{error_msg}")
            print("\n[INFO] Dicas de troubleshooting:")
            print("   1. Verifique se a API key está configurada")
            print("   2. Simplifique a pergunta")
            print("   3. Verifique os logs acima para mais detalhes")
        
        return error_msg


print("\n[OK] Função helper 'ask_growth_agent' disponível!")
print("\n[INFO] Exemplo de uso:")
print('   response = await ask_growth_agent("Quais métricas devo acompanhar?")')
print('   print(response)')

In [ ]:
# ============================================================================
# CELL 17: EXEMPLOS DE USO AVANÇADO
# ============================================================================

print("\n" + "="*70)
print("EXEMPLOS DE USO AVANÇADO")
print("="*70)

print("""
# 1. ANÁLISE DE FUNIL
query = '''
Tenho os seguintes dados de funil:
- 10.000 visitas à landing page
- 3.500 signups (35%)
- 700 purchases (20% dos signups, 7% do total)

Qual o principal gargalo e como resolvê-lo?
'''
response = await ask_growth_agent(query)

# 2. DESIGN DE EXPERIMENTO
query = '''
Quero testar se adicionar social proof (reviews) na página de checkout
aumenta a conversão de 2.5% para 3.0%.

Desenhe o experimento A/B completo.
'''
response = await ask_growth_agent(query)

# 3. ANÁLISE DE DADOS CSV
query = f'''
Analise estes dados CSV e identifique oportunidades de growth:

{csv_data}

Forneça insights acionáveis.
'''
response = await ask_growth_agent(query)

# 4. VALIDAÇÃO ESTATÍSTICA
query = '''
Rodei um teste A/B com estes resultados:
- Controle: 250 conversões em 10.000 usuários (2.5%)
- Tratamento: 280 conversões em 10.000 usuários (2.8%)

O resultado é estatisticamente significativo? Devo implementar?
'''
response = await ask_growth_agent(query)

# 5. PRIORIZAÇÃO DE EXPERIMENTOS
query = '''
Tenho 3 ideias de experimentos:
1. Mudar cor do botão (fácil, impacto médio)
2. Simplificar formulário de signup (médio, impacto alto)
3. Adicionar onboarding tutorial (difícil, impacto alto)

Como priorizar usando RICE framework?
'''
response = await ask_growth_agent(query)
""")

print("\n[INFO] Cole qualquer exemplo acima em uma nova célula e execute!")


In [ ]:
# ============================================================================
# CELL 18: SUMÁRIO FINAL E MÉTRICAS
# ============================================================================

print("\n" + "="*70)
print("SUMÁRIO FINAL - SISTEMA GROWTH AGENTS")
print("="*70)

summary = {
    "status": "OPERATIONAL",
    "model": MODEL,
    "agents": {
        "coordinator": "CoordinatorAgent (root)",
        "specialists": [
            "FunnelAgent",
            "StatsAgent", 
            "InsightsAgent",
            "DataQualityAgent",
            "TrackingAgent",
            "ExperimentAgent"
        ],
        "composite": [
            "ParallelDiagnosticAgent",
            "SequentialAnalysisPipeline",
            "ExperimentRefinementLoop"
        ]
    },
    "tools": {
        "function_tools": [
            "calculate_sample_size",
            "calculate_statistical_significance",
            "analyze_csv_dataframe"
        ],
        "builtin_tools": ["google_search"],
        "bigquery": "enabled" if bq_toolset else "disabled",
        "code_execution": "enabled" if CODE_EXECUTION_AVAILABLE else "disabled"
    },
    "features": {
        "parallel_execution": True,
        "sequential_pipeline": True,
        "iterative_refinement": True,
        "output_keys": True,
        "error_handling": True,
        "structured_logging": True
    },
    "architecture": "100% MDS Partner Implementation"
}

print("\n[OK] SISTEMA OPERACIONAL")
print(json.dumps(summary, indent=2))

print("\n" + "="*70)
print("PRONTO PARA USO!")
print("="*70)
print("""
[INFO] Próximos passos:

1. Execute uma consulta simples:
   response = await ask_growth_agent("Como calcular sample size para A/B test?")
   
2. Analise seus dados:
   - Prepare CSV com eventos de usuários
   - Use analyze_csv_dataframe para exploração
   - Chame FunnelAgent para análise completa

3. Desenhe experimentos:
   - Use ExperimentAgent para planos detalhados
   - Valide com StatsAgent
   - Priorize com InsightsAgent

4. Para produção:
   - Configure BigQuery para dados reais
   - Adicione métricas de observabilidade
   - Implemente testes automatizados

[OK] Sistema pronto! Happy experimenting!
""")

logger.info("Sistema Growth Agents inicializado com sucesso")
logger.info("Todas as células executadas sem erros críticos")
print("\n[OK] Notebook completo executado com sucesso!")
print("="*70)